<a href="https://colab.research.google.com/github/dongukmoon/misc/blob/main/vllm_quickstart_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# vLLM Tutorial: Quick Start on Google Colab
This guide walks you through setting up and running offline batched inference and OpenAI-compatible inference with vLLM on Google Colab.


1.   Offline batched inference
2.   OpenAI-compatible inference


## Running Offline Batched Inference

**Step 1: Install Required Packages**

Start by installing vLLM and other necessary packages using pip:

In [ ]:
!pip install vllm torch  triton

**Step 2: Set Up GPU Environment**

Google Colab offers free access to a Tesla T4 GPU (15GB RAM), which is sufficient for running vLLM. In this tutorial, we'll use the GPT-2 model and set the `gpu_memory_utilization` parameter to 0.5. This ensures that the key-value cache reserved for model inference does not exceed 50% of the total GPU memory. Without this setting, you might encounter out-of-memory errors due to the large key-value cache required for the maximum number of tokens supported by GPT-2.
With `gpu_memory_utilization`=0.5, the model weights will use about 0.24GB, and the key-value cache will require approximately 6.52GB of GPU memory.



**Step 3: Load the Model and Generate Outputs**

Below is the Python script to load the GPT-2 model, define prompts, and generate text outputs:

In [ ]:
from vllm import LLM, SamplingParams

# Load a pre-trained Hugging Face model
model_name = "gpt2"  # Replace with your desired model

# Create an LLM.
#gpu_memory_utilization: The ratio (between 0 and 1) of GPU memory to
#reserve for the model weights, activations, and KV cache. Higher
#values will increase the KV cache size and thus improve the model's
#throughput. However, if the value is too high, it may cause out-of-
# memory (OOM) errors.
llm = LLM(model_name, gpu_memory_utilization = 0.5)

# Sample prompts.
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")


**Step 4: Monitor GPU Memory Usage**

Use the nvidia-smi command to check GPU memory usage. As shown, approximately 7.5GB out of 15GB of GPU memory is used.

In [ ]:
!nvidia-smi

**Sample Output**

The output of the script may look like this:

```
Prompt: 'Hello, my name is', Generated text: ' Scott. If you want to talk to me, you can. I am a'
Prompt: 'The president of the United States is', Generated text: ' a trained lawyer with experience in working with the vast majority of lawyers who are part'
Prompt: 'The capital of France is', Generated text: ' on the basis of the basis of the shared law. This is called the Common'
Prompt: 'The future of AI is', Generated text: ' becoming a bit more difficult. The big, big picture lies with AI. We'
```

**Step 5: Handle GPU Memory for Repeated Runs**

If you run the script again without restarting the Colab instance, you might encounter out-of-memory errors. To free up GPU memory, you can simply restart the Colab environment using the following command:

In [ ]:
import os
os._exit(00)

## Running OpenAI-Compatible Inference
vLLM can act as a server implementing the OpenAI API protocol, making it a drop-in replacement for applications using OpenAI APIs. The default server address is http://localhost:8000. You can customize it with the --host and --port arguments. The server hosts one model at a time and supports endpoints such as list models, create completion, and create chat completion.

**Starting the vLLM Server**

Run the following command to start the vLLM server with the Qwen/Qwen2.5-1.5B-Instruct model:


In [ ]:
# Cannot use FlashAttention-2 backend for Volta and Turing GPUs
# and T4 does not support BF16, causing error.
# So, I used a L4 GPU instance

# wait until the server bootup is complete. Check the log in 'nohup.out'
!nohup vllm serve Qwen/Qwen2.5-1.5B-Instruct --port 8001 --gpu-memory-utilization 0.3 &


  This server can be queried in the same format as OpenAI API. For example, to list the models:

In [ ]:
!curl http://localhost:8001/v1/models

Once your server is started, you can query the model with input prompts:

In [ ]:
!curl http://localhost:8001/v1/completions \
-H "Content-Type: application/json" \
-d '{"model": "Qwen/Qwen2.5-1.5B-Instruct","prompt": "San Francisco is a", \
"max_tokens": 7, \
"temperature": 0 \
}'

**Using the OpenAI Python Package**

vLLM's API server is OpenAI-compatible, allowing you to use the OpenAI Python package:

**Text Completion**



In [ ]:
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8001/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model="Qwen/Qwen2.5-1.5B-Instruct",
                                      prompt="San Francisco is a")
print("Completion result:", completion)

**Chat Completion**

vLLM also supports the OpenAI Chat Completions API, enabling a dynamic and interactive way to communicate with the model. Through the chat interface, users can engage in back-and-forth exchanges, with context preserved via chat history. This feature is particularly useful for tasks that demand continuity, detailed explanations, or maintaining context across multiple interactions.

In [ ]:
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8001/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

chat_response = client.chat.completions.create(
    model="Qwen/Qwen2.5-1.5B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a joke."},
    ]
)
print("Chat response:", chat_response)

# References

*   https://docs.vllm.ai/en/latest/getting_started/quickstart.html

*   https://docs.vllm.ai/en/stable/models/engine_args.html